In [ ]:
!pip install -U langchain langgraph google-api-python-client langchain[google-genai]


In [ ]:
import os

def _set_env(key):
    value = os.environ.get(key)
    if value is None:
        value = input(f"Please enter your {key}: ")
        os.environ[key] = value

# Set keys
_set_env("GOOGLE_API_KEY")   # From Google Cloud Console
_set_env("GOOGLE_CSE_ID")    # From Programmable Search Engine (CSE)


In [ ]:
from langchain.tools import Tool
from googleapiclient.discovery import build

def google_search(query: str, max_results: int = 3):
    service = build("customsearch", "v1", developerKey=os.environ["GOOGLE_API_KEY"])
    res = service.cse().list(q=query, cx=os.environ["GOOGLE_CSE_ID"], num=max_results).execute()
    results = res.get("items", [])
    return "\n".join([f"{item['title']}: {item['link']}" for item in results])

# Wrap as a LangChain tool
google_tool = Tool(
    name="GoogleSearch",
    func=google_search,
    description="Use this tool to search the web using Google Search"
)

tools = [google_tool]


In [ ]:
from langchain.chat_models import init_chat_model

# If not already set
os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY")

# Initialize Gemini model
llm = init_chat_model("google_genai:gemini-2.0-flash")
llm_with_tools = llm.bind_tools(tools)


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)


In [ ]:
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}

tool_node = BasicToolNode(tools=[google_tool])
graph_builder.add_node("tools", tool_node)


In [ ]:
def route_tools(state: State):
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError("No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END

graph_builder.add_conditional_edges("chatbot", route_tools, {"tools": "tools", END: END})
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()


In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        user_input = "What is LangGraph?"
        print("User:", user_input)
        stream_graph_updates(user_input)
        break
